# Переобучение (Overfitting) & Недообучение (Underfitting) & Train/Test/CV

---

**Источники:**

[Переобучение](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9F%D0%B5%D1%80%D0%B5%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5)

[Машинное обучение в 10 диаграммах](http://datareview.info/dlya-it-spetsialista/mashinnoe-obuchenie-v-10-diagrammah/)

[Overfitting and Underfitting With Machine Learning Algorithms](https://machinelearningmastery.com/overfitting-and-underfitting-with-machine-learning-algorithms/)

[Training, validation, and test sets](https://en.wikipedia.org/wiki/Training,_validation,_and_test_sets)

[Train/Test Split and Cross Validation in Python](https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6)

[Overfitting](https://en.wikipedia.org/wiki/Overfitting#:~:text=In%20statistics%2C%20overfitting%20is%20%22the,or%20predict%20future%20observations%20reliably%22.)

[Смещение (bias) и разброс (variance)](https://dyakonov.org/2018/04/25/%D1%81%D0%BC%D0%B5%D1%89%D0%B5%D0%BD%D0%B8%D0%B5-bias-%D0%B8-%D1%80%D0%B0%D0%B7%D0%B1%D1%80%D0%BE%D1%81-variance-%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8-%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82/)

[Кросс-валидация (Cross-validation)](https://wiki.loginom.ru/articles/cross-validation.html)

[Кросс-валидация](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9A%D1%80%D0%BE%D1%81%D1%81-%D0%B2%D0%B0%D0%BB%D0%B8%D0%B4%D0%B0%D1%86%D0%B8%D1%8F)

[Cross-validation: evaluating estimator performance](https://scikit-learn.org/stable/modules/cross_validation.html)

---

In [1]:
# ВНИМАНИЕ: необходимо удостовериться, что виртуальная среда выбрана правильно!

# Для MacOS/Ubuntu
# !which pip

# Для Windows
# !where pip

In [2]:
# !conda install pandas numpy scikit-learn -y

In [3]:
import numpy as np

np.__version__

'1.19.2'

In [4]:
import pandas as pd

pd.__version__

'1.2.3'

In [5]:
import sklearn

sklearn.__version__


'0.24.1'

## Загрузка данных

[Источник (Bike Share Daily Data)](https://www.kaggle.com/contactprad/bike-share-daily-data).

In [6]:
df = pd.read_csv('./../../data/bike_sharing_daily.csv', 
                 index_col=0)
df

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
instant,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,247,1867,2114
728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095
729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,159,1182,1341


## Overfitting (Переобучение) & Underfitting (Недообучение)

**Обобщающая способность (generalization ability, generalization performance)**.

Говорят, что алгоритм обучения обладает **способностью к обобщению**, если вероятность ошибки на тестовой выборке достаточно мала или хотя бы предсказуема, то есть не сильно отличается от ошибки на обучающей выборке. **Обобщающая способность** тесно связана с понятиями **переобучения** и **недообучения**.


<img src="images/overtraining.svg"/>

<img src="images/overfit_underfit_normal.jpg"/>

### Переобучение (Overfitting)

**Переобучение (переподгонка, пере- в значении "слишком", англ. overfitting, overtraining)** в машинном обучении и статистике — явление, когда построенная модель хорошо объясняет примеры из обучающей выборки, но относительно плохо работает на примерах, не участвовавших в обучении (на примерах из тестовой выборки).

Это связано с тем, что при построении модели ("в процессе обучения") в обучающей выборке обнаруживаются некоторые случайные закономерности, которые отсутствуют в генеральной совокупности.

Иными словами — модель запоминает огромное количество всех возможных примеров вместо того, чтобы научиться подмечать особенности.

Способы борьбы с переобучением зависят от метода моделирования и способа построения модели.

Существуют **методы предотвращения переобучения**, например, перекрёстная проверка (кросс-проверка, скользящий контроль, англ. **cross-validation**).

### Недообучение (Underfitting)

**Недообучение (Underfitting)** — прекращение процесса обучения аналитической обучаемой модели ранее, чем она достигнет состояния, которое обеспечит достаточно малую ошибку на обучающем множестве.

**Недообучение** может возникнуть по различным причинам. Например, пользователь может задать неудачные условия остановки обучения (число итераций, значение ошибки обучения и т.д.).

Кроме этого, возможно, что конфигурация модели, не соответствующая решаемой задаче, или неудачная обучающая выборка просто не позволят алгоритму приблизиться к оптимуму.

## Training / Test / Cross-validation

**Кросс-валидация Cross-validation или скользящий контроль** — процедура эмпирического оценивания обобщающей способности алгоритмов.

С помощью **кросс-валидации** эмулируется наличие тестовой выборки, которая не участвует в обучении, но для которой известны правильные ответы.

**k-fold кросс-валидация**:
1. Обучающая выборка разбивается на k непересекающихся одинаковых по объему частей;
2. Производится k итераций. На каждой итерации происходит следующее:
    1. Модель обучается на k−1 части обучающей выборки;
    1. Модель тестируется на части обучающей выборки, которая не участвовала в обучении.
Каждая из k частей единожды используется для тестирования. Как правило, k=10 (5 в случае малого размера выборки).

**Кросс-валидация имеет два основных преимущества** перед применением одного множества для обучения и одного для тестирования модели:
- Распределение классов оказывается более равномерным, что улучшает качество обучения.
- Если при каждом проходе оценить выходную ошибку модели и усреднить ее по всем проходам, то полученная оценка будет более достоверной.

На практике чаще всего выбирается `k=10` (10-ти проходная перекрестная проверка), когда модель обучается на 9/10 данных и тестируется на 1/10. Исследования показали, что в этом случае получается наиболее достоверная оценка выходной ошибки модели.

<img src="images/pipeline.png" height="500" width="500"/>

<img src="images/train_test_valid.png" height="600" width="600">

<img src="images/grid_search_cross_validation.png" height="600" width="600">

In [8]:
from sklearn.model_selection import train_test_split
X = df.drop('cnt',axis=1)
y = df['cnt']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [9]:
X_train.head(5)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
instant,,,,,,,,,,,,,,
643,2012-10-04,4,1,10,0,4,1,2,0.657500,0.607975,0.722917,0.117546,891,6437
706,2012-12-06,4,1,12,0,4,1,1,0.255833,0.258204,0.508750,0.174754,340,5035
567,2012-07-20,3,1,7,0,5,1,2,0.665833,0.613025,0.844167,0.208967,747,5123
228,2011-08-16,3,0,8,0,2,1,1,0.700833,0.645837,0.578333,0.236329,721,4004
271,2011-09-28,4,0,9,0,3,1,2,0.635000,0.575158,0.848750,0.148629,480,3427


In [10]:
X_test.head(5)

,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered
instant,,,,,,,,,,,,,,
489,2012-05-03,2,1,5,0,4,1,2,0.560000,0.537242,0.768333,0.133696,764,5657
422,2012-02-26,1,1,2,0,0,0,1,0.279167,0.268308,0.410000,0.205229,515,2874
92,2011-04-02,2,0,4,0,6,0,2,0.315000,0.315637,0.653750,0.197146,898,1354
301,2011-10-28,4,0,10,0,5,1,2,0.330833,0.318812,0.585833,0.229479,456,3291
178,2011-06-27,3,0,6,0,1,1,2,0.682500,0.637004,0.658333,0.107588,854,3854


In [11]:
y_train.head(5)

instant
643    7328
706    5375
567    5870
228    4725
271    3907
Name: cnt, dtype: int64

In [12]:
y_test.head(5)

instant
489    6421
422    3389
92     2252
301    3747
178    4708
Name: cnt, dtype: int64

## Регуляризация (Regularization)
TODO

### L1

### L2

### Elastic Net